In [1]:
import pandas as pd
import numpy as np

In [2]:
search_trend_url ="https://raw.githubusercontent.com/google-research/open-covid-19-data/master/data/exports/search_trends_symptoms_dataset/United%20States%20of%20America/2020_US_weekly_symptoms_dataset.csv"
hospitalization_url ="https://raw.githubusercontent.com/google-research/open-covid-19-data/master/data/exports/cc_by/aggregated_cc_by.csv"
search_trend_df = pd.read_csv(search_trend_url)
hospitalization_df = pd.read_csv(hospitalization_url)

In [3]:
search_trend_df = search_trend_df.drop(search_trend_df.columns[[0,1,2,4,5,6]], axis=1)
search_trend_df = search_trend_df.dropna(axis=1, how='all')
search_trend_df = search_trend_df.rename(columns={'sub_region_1':'region_name'})

In [4]:
hospitalization_df = hospitalization_df[['open_covid_region_code','region_name','date','hospitalized_new']]
hospitalization_df = hospitalization_df[hospitalization_df['open_covid_region_code'].str.contains('^US-')]
hospitalization_df = hospitalization_df.drop(hospitalization_df.columns[[0]], axis=1)
regions = hospitalization_df.region_name.unique()

for region in regions:
    region_df = hospitalization_df[hospitalization_df['region_name'] == region]
    if (region_df['hospitalized_new'] == 0).all():
        hospitalization_df = hospitalization_df.drop(hospitalization_df[hospitalization_df.region_name == region].index, axis=0) 

In [5]:
hospitalization_df['date'] = pd.to_datetime(hospitalization_df['date'], format='%Y-%m-%d')
hospitalization_df = hospitalization_df.groupby(['region_name']).resample('W', on='date',loffset='1d').sum()
hospitalization_df = hospitalization_df.reset_index()

search_trend_df['date'] = pd.to_datetime(search_trend_df['date'], format='%Y-%m-%d')
result_df = search_trend_df.merge(hospitalization_df, on=['region_name', 'date'])
print(len(result_df.region_name.unique()))
result_df.to_csv('result.csv', index=False)

14
